### Notebook to genereate time series from TC output

Will also mask out beyond TC area

Assumes output is in a single netcdf file on pressure levels.

James Ruppert  
jruppert@ou.edu  
September 2022

In [9]:
from netCDF4 import Dataset
import numpy as np
import matplotlib
matplotlib.use('pdf')
import matplotlib.pyplot as plt
# import matplotlib.colors as colors
from matplotlib import ticker, cm
import subprocess
from mask_tc_track import mask_tc_track
import sys
import pandas as pd

#### Settings

In [2]:
# storms=['haiyan','maria']
#storms=['maria']
storm = 'haiyan'
# storm = 'maria'
# nstorm = np.size(storms)

# Strat/Conv index subset
istrat=1 # Convective
# istrat=2 # Stratiform
istrat=4 # Convective/Stratiform fraction

# TC tracking
ptrack='600' # tracking pressure level
var_track = 'rvor' # variable
rmax = 8 # radius (deg) limit for masking around TC center

In [3]:
# Strat/Conv index subset
if istrat == -1:
    fig_extra=''
else:
    if istrat == 0:
        strattag='Nonrain'
    elif istrat == 1:
        strattag='Conv'
    elif istrat == 2:
        strattag='Strat'
    elif istrat == 3:
        strattag='Anv'
    elif istrat == 4:
        strattag='frac'
    fig_extra='_'+strattag.lower()

def get_tshift(itest):
    if itest == 'ctl':
        tshift=0
    elif itest == 'ncrf36h':
        tshift=36
    elif itest == 'ncrf48h':
        tshift=48
    return tshift

#### Main read loop

In [4]:
# for istorm in range(nstorm):

    # storm = storms[istorm]

# Tests to read and compare
if storm == 'haiyan':
    tests = ['ctl','ncrf36h']
elif storm == 'maria':
#        tests = ['ctl','ncrf36h']
    tests = ['ctl','ncrf48h']

# How many members
nmem = 10 # number of ensemble members (1-5 have NCRF)
nmem = 2
# nmem = 1
# Starting member to read
memb0=1

# #### Directories

figdir = "/home/jamesrup/figures/tc/ens/"
main = "/ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/tc_ens/"

nums=np.arange(memb0,nmem+memb0,1)
nums=nums.astype(str)
nustr = np.char.zfill(nums, 2)
memb_all=np.char.add('memb_',nustr)


##### Get dimensions

datdir = main+storm+'/'+memb_all[0]+'/'+tests[0]+'/'
varfil_main = Dataset(datdir+'post/d02/T.nc')
nz = varfil_main.dimensions['level'].size
nx1 = varfil_main.dimensions['lat'].size
nx2 = varfil_main.dimensions['lon'].size#-xmin-1
pres = varfil_main.variables['pres'][:] # hPa
varfil_main.close()

process = subprocess.Popen(['ls '+datdir+'/wrfout_d02_*'],shell=True,
    stdout=subprocess.PIPE,universal_newlines=True)
output = process.stdout.readline()
wrffil = output.strip() #[3]
varfil_main = Dataset(wrffil)
lat = varfil_main.variables['XLAT'][:][0] # deg
lon = varfil_main.variables['XLONG'][:][0] # deg
varfil_main.close()
lon1d=lon[0,:]
lat1d=lat[:,0]


for imemb in range(nmem):

    print('Running imemb: ',memb_all[imemb])


    # First test

    itest = tests[0]
    tshift1 = get_tshift(itest)

    datdir = main+storm+'/'+memb_all[imemb]+'/'+itest+'/'
    track_file = datdir+'track_'+var_track+'_'+ptrack+'hPa.nc'

    # Read variable
    datdir = main+storm+'/'+memb_all[imemb]+'/'+itest+'/post/d02/'
    varfil_main = Dataset(datdir+'strat.nc')
    strat = varfil_main.variables['strat'][:,:,:,:] # 0-non-raining, 1-conv, 2-strat, 3-other/anvil
    varfil_main.close()
    nt = strat.shape[0]

    t0_test1=0
    t1_test1=nt

    # Mask out around TC center
    strat = mask_tc_track(track_file, rmax, strat, lon, lat, t0_test1, t1_test1)
    count_total = np.ma.MaskedArray.count(strat, axis=(1,2,3))

    # Mask out based on strat/conv
    if istrat == 4:
        strat_ind = np.ma.masked_where((strat != 2), strat)
        conv_ind = np.ma.masked_where((strat != 1), strat)
        count_strat = np.ma.MaskedArray.count(strat_ind, axis=(1,2,3))
        count_conv = np.ma.MaskedArray.count(conv_ind, axis=(1,2,3))
        frac_strat = count_conv / count_strat
    else:
        strat = np.ma.masked_where((strat != istrat), strat, copy=True)
        count_strat = np.ma.MaskedArray.count(strat, axis=(1,2,3))
        frac_strat = count_strat / count_total

    # Plot variable
    # plt.plot(range(t0_test1+tshift1,t1_test1+tshift1), frac_strat, linewidth=1, 
    #     label=nustr[imemb], color=color_t1, linestyle='solid')

    if imemb == 0:
        frac_strat_all_t1 = np.reshape(frac_strat, (nt,1))
    else:
        frac_strat_all_t1 = np.append(frac_strat_all_t1, np.reshape(frac_strat, (nt,1)), axis=1)


    # Second test

    itest = tests[1]
    tshift2 = get_tshift(itest)

    datdir = main+storm+'/'+memb_all[imemb]+'/'+itest+'/'
    track_file = datdir+'track_'+var_track+'_'+ptrack+'hPa.nc'

    # Read variable
    datdir = main+storm+'/'+memb_all[imemb]+'/'+itest+'/post/d02/'
    varfil_main = Dataset(datdir+'strat.nc')
    strat = varfil_main.variables['strat'][:,:,:,:] # 0-non-raining, 1-conv, 2-strat, 3-other/anvil
    varfil_main.close()
    nt = strat.shape[0]

    t0_test2=0
    t1_test2=nt

    # Mask out where TC track is undefined


    # Mask out around TC center
    strat = mask_tc_track(track_file, rmax, strat, lon, lat, t0_test2, t1_test2)
    count_total = np.ma.MaskedArray.count(strat, axis=(1,2,3))

    # Mask out based on strat/conv
    if istrat == 4:
        strat_ind = np.ma.masked_where((strat != 2), strat)
        conv_ind = np.ma.masked_where((strat != 1), strat)
        count_strat = np.ma.MaskedArray.count(strat_ind, axis=(1,2,3))
        count_conv = np.ma.MaskedArray.count(conv_ind, axis=(1,2,3))
        frac_strat = count_conv / count_strat
    else:
        strat = np.ma.masked_where((strat != istrat), strat, copy=True)
        count_strat = np.ma.MaskedArray.count(strat, axis=(1,2,3))
        frac_strat = count_strat / count_total

    # Plot variable
    # plt.plot(range(t0_test2+tshift2,t1_test2+tshift2), frac_strat, linewidth=1,
    #     label=nustr[imemb], color=color_t2, linestyle='--')

    if imemb == 0:
        frac_strat_all_t2 = np.reshape(frac_strat, (nt,1))
    else:
        frac_strat_all_t2 = np.append(frac_strat_all_t2, np.reshape(frac_strat, (nt,1)), axis=1)


Running imemb:  memb_01


/tmp/ipykernel_54785/3455581084.py:85: RuntimeWarning: invalid value encountered in divide
  frac_strat = count_conv / count_strat


Running imemb:  memb_02


In [7]:
tmp = frac_strat_all_t2[:,1]
print(tmp)

[0.51869205 0.54702388 0.6327698  0.64102564 0.67247429 0.61317845
 0.626762   0.631847   0.67962149 0.778157   0.76860927 0.67486631
 0.65244845 0.60374111 0.62003292 0.657014   0.64176725 0.64295602
 0.59781384 0.54973646 0.48715278 0.4379976  0.45851907 0.42068966
 0.41406926]


In [41]:
# tmp_smooth = pd.DataFrame.rolling(window=3, center=True).mean()
tmp2 = np.copy(frac_strat_all_t2)
tmpdf = pd.DataFrame(tmp2)
tmp_smooth = tmpdf.rolling(window=3, center=True, closed='both', axis=0).mean()

# print( tmpdf.rolling(window=3, center=True, closed='both').mean() )
print(tmpdf[0][:])
# print(tmp_smooth[1][:])
# print( tmpdf )

           0         1
0   0.602809  0.518692
1   0.508872  0.547024
2   0.515178  0.632770
3   0.568531  0.641026
4   0.688843  0.672474
5   0.731857  0.613178
6   0.704769  0.626762
7   0.638509  0.631847
8   0.605608  0.679621
9   0.592065  0.778157
10  0.537010  0.768609
11  0.501365  0.674866
12  0.530571  0.652448
13  0.597047  0.603741
14  0.663531  0.620033
15  0.717748  0.657014
16  0.694266  0.641767
17  0.636739  0.642956
18  0.673703  0.597814
19  0.661522  0.549736
20  0.530068  0.487153
21  0.483650  0.437998
22  0.500572  0.458519
23  0.538190  0.420690
24  0.574956  0.414069


---
### Plotting routines

In [1]:
# Create figure with all tracks

# ### Plotting routines ##############################################

font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)

# ### Combined plot ##############################################

color_t1 = 'red'
color_t2 = 'blue'

# create figure
fig = plt.figure(figsize=(14,5))
ax = fig.add_subplot(111)

ax.set_title(storm.capitalize()+': '+strattag.capitalize(), fontsize=20)
ax.set_ylabel('Fraction')
ax.set_xlabel('Time [hours]')

# Plot individual members
# for imemb in range(nmem):
#     plt.plot(range(t0_test1+tshift1,t1_test1+tshift1), frac_strat_all_t1[:,imemb], linewidth=1, 
#         label=nustr[imemb], color=color_t1, linestyle='solid')

# Plot means and standard deviations
# frac_mean_t1 = np.mean(frac_strat_all_t1, axis=1, where=np.where(np.isfinite(frac_strat_all_t1)))
frac_mean_t1 = np.nanmean(frac_strat_all_t1, axis=1)
frac_std_t1 = np.nanstd(frac_strat_all_t1, axis=1)
mean_pstd_t1 = frac_mean_t1 + frac_std_t1
mean_mstd_t1 = frac_mean_t1 - frac_std_t1

plt.plot(range(t0_test1 + tshift1, t1_test1 + tshift1), frac_mean_t1, 
    linewidth=2, label=tests[0].upper(), color=color_t1, linestyle='solid')
plt.fill_between(range(t0_test1 + tshift1, t1_test1 + tshift1), mean_pstd_t1, mean_mstd_t1, alpha=0.2,
    color=color_t1)

# Plot individual members
# for imemb in range(nmem):
#     plt.plot(range(t0_test2+tshift2,t1_test2+tshift2), frac_strat_all_t2[:,imemb], linewidth=1,
#         label=nustr[imemb], color=color_t2, linestyle='solid')

# frac_mean_t2 = np.mean(frac_strat_all_t2, axis=1)
frac_mean_t2 = np.nanmean(frac_strat_all_t2, axis=1)
frac_std_t2 = np.nanstd(frac_strat_all_t2, axis=1)
mean_pstd_t2 = frac_mean_t2 + frac_std_t2
mean_mstd_t2 = frac_mean_t2 - frac_std_t2

plt.plot(range(t0_test2 + tshift2, t1_test2 + tshift2), frac_mean_t2, 
    linewidth=2, label=tests[1].upper(), color=color_t2, linestyle='--')
plt.fill_between(range(t0_test2 + tshift2, t1_test2 + tshift2), mean_pstd_t2, mean_mstd_t2, alpha=0.2,
    color=color_t2)

# plt.legend(loc="upper right")

plt.show()
# plt.savefig(figdir+'tser_'+storm+fig_extra+'_'+tests[1]+'.png',dpi=200, facecolor='white', \
#             bbox_inches='tight', pad_inches=0.2)
# plt.close()

NameError: name 'matplotlib' is not defined